In [ ]:
import yaml
import joblib


def dump(value=None, filename=None):
    if (value is not None) and (filename is not None):
        joblib.dump(value=value, filename=filename)

    else:
        raise ValueError("The value and filename are required".capitalize())


def load(filename=None):
    if filename is not None:
        return joblib.load(filename=filename)

    else:
        raise ValueError("The filename is required".capitalize())


def config():
    with open("./config.yml", "r") as file:
        return yaml.safe_load(file)

In [ ]:
import os
import torch
import argparse
import pandas as pd
from utils import dump, load, config
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


class Loader:
    def __init__(self, dataset=None, batch_size=64, split_size=0.25):
        self.datframe = dataset
        self.batch_size = batch_size
        self.split_size = split_size

        self.PROCESSED_PATH = config()["path"]["PROCESSED_PATH"]

    def normalized_dataset(self, dataset):
        if isinstance(dataset, torch.Tensor):
            scaler = StandardScaler()
            return torch.tensor(scaler.fit_transform(dataset))
        else:
            raise TypeError("The dataset must be a torch tensor")

    def preprocessing(self):
        self.dataset = pd.read_csv(self.datframe)

        if isinstance(self.dataset, pd.DataFrame):
            if "id" in self.dataset.columns:
                self.dataset.drop(["id"], axis=1, inplace=True)
            else:
                print("The dataset does not have an id column")

            if "diagnosis" in self.dataset.columns:
                if (
                    "M" in self.dataset["diagnosis"].unique()
                    and "B" in self.dataset["diagnosis"].unique()
                ):
                    self.dataset["diagnosis"] = self.dataset["diagnosis"].map(
                        {"B": 0, "M": 1}
                    )
                else:
                    print("The dataset does not have the correct diagnosis values")
            else:
                print("The dataset does not have a diagnosis column")

            X = self.dataset.iloc[:, 1:]
            y = self.dataset.iloc[:, 0]

            y = y.astype(int)

            X = torch.tensor(data=X.values, dtype=torch.float)
            y = torch.tensor(data=y.values, dtype=torch.long)

            X = self.normalized_dataset(dataset=X)

            return {"X": X, "y": y}

        else:
            raise TypeError("The dataset must be a pandas DataFrame")

    def create_dataloader(self):
        data = self.preprocessing()

        if isinstance(data["X"], torch.Tensor) and isinstance(data["y"], torch.Tensor):
            X = data["X"]
            y = data["y"]

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.split_size, random_state=42
            )

            train_dataloader = DataLoader(
                dataset=list(zip(X_train, y_train)),
                batch_size=self.batch_size,
                shuffle=True,
            )

            test_dataloader = DataLoader(
                dataset=list(zip(X_test, y_test)),
                batch_size=self.batch_size,
                shuffle=True,
            )

            os.makedirs(self.PROCESSED_PATH, exist_ok=True)
            for filename, value in [
                ("train_dataloader", train_dataloader),
                ("test_dataloader", test_dataloader),
            ]:
                dump(
                    value=value,
                    filename=os.path.join(self.PROCESSED_PATH, f"{filename}.pkl"),
                )

            print(
                "The dataloader has been saved in the folder {}".format(
                    self.PROCESSED_PATH
                )
            )

    @staticmethod
    def dataset_details():
        PROCESSED_PATH = config()["path"]["PROCESSED_PATH"]
        FILES_PATH = config()["path"]["FILES_PATH"]

        if os.path.exists(PROCESSED_PATH):
            train_dataloader = load(
                filename=os.path.join(PROCESSED_PATH, "train_dataloader.pkl")
            )
            test_dataloader = load(
                filename=os.path.join(PROCESSED_PATH, "test_dataloader.pkl")
            )

            X, y = next(iter(train_dataloader))

        pd.DataFrame(
            {
                "Train Data(Total)": sum(X.size(0) for X, y in train_dataloader),
                "Test Data(Total)": sum(X.size(0) for X, y in test_dataloader),
                "Data Shpe (Train)": str(X.size()),
            },
            index=["Quantity"],
        ).T.to_csv(
            os.path.join(FILES_PATH, "dataset_details.csv"),
        )


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Data Loader for the Cancer".capitalize()
    )
    parser.add_argument(
        "--dataset",
        default=os.path.join(config()["path"]["RAW_PATH"], "breast-cancer.csv"),
        help="Defin the dataset".capitalize(),
    )
    parser.add_argument(
        "--split_size",
        default=config()["data"]["split_size"],
        help="Defin the split size".capitalize(),
    )
    parser.add_argument(
        "--batch_size",
        default=config()["data"]["batch_size"],
        help="Defin the batch size".capitalize(),
    )

    args = parser.parse_args()

    loader = Loader(
        dataset=args.dataset,
        batch_size=args.batch_size,
        split_size=args.split_size,
    )

    loader.create_dataloader()

    loader.dataset_details()

In [ ]:
import os
import torch
import argparse
import torch.nn as nn
from utils import config
from torchsummary import summary
from torchview import draw_graph


class Model(nn.Module):
    def __init__(self, in_features=30):
        super(Model, self).__init__()

        self.in_features = in_features
        self.out_features = 128

        self.layers = []

        for _ in range(4):
            self.layers.append(
                nn.Sequential(
                    nn.Linear(
                        in_features=self.in_features, out_features=self.out_features
                    ),
                    nn.ReLU(),
                    nn.BatchNorm1d(num_features=self.out_features),
                )
            )

            self.in_features = self.out_features
            self.out_features = self.out_features // 2

        self.model = nn.Sequential(*self.layers)

        self.out = nn.Sequential(
            nn.Linear(in_features=self.in_features, out_features=1), nn.Sigmoid()
        )

    def forward(self, x):
        if isinstance(x, torch.Tensor):
            x = self.model(x)
            return self.out(x)

        else:
            raise ValueError("Input must be a torch.Tensor".capitalize())


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Model for breast Cancer".title())
    parser.add_argument(
        "--model",
        action="store_true",
        default=None,
        help="Model for Breast Cancer".capitalize(),
    )
    parser.add_argument(
        "--in_features",
        default=config()["model"]["in_features"],
        type=int,
        help="Define the number of features".capitalize(),
    )

    args = parser.parse_args()

    if args.model:

        model = Model(in_features=args.in_features)

        summary(model=model, input_size=(30,))

        draw_graph(model=model, input_data=torch.randn(32, 30)).visual_graph.render(
            filename=os.path.join(config()["path"]["FILES_PATH"], "Model"),
            format="jpeg",
        )

In [ ]:
import os
import torch.nn as nn
from model import Model
import torch.optim as optim
from utils import load, config


def load_dataloader():
    processed_path = config()["path"]["PROCESSED_PATH"]

    if os.path.exists(processed_path):
        train_dataloader = load(
            filename=os.path.join(processed_path, "train_dataloader.pkl")
        )

        test_dataloader = load(
            filename=os.path.join(processed_path, "test_dataloader.pkl")
        )

        return {
            "train_dataloader": train_dataloader,
            "test_dataloader": test_dataloader,
        }


def helpers(**kwargs):
    lr = kwargs["lr"]
    adam = kwargs["adam"]
    SGD = kwargs["SGD"]
    beta1 = kwargs["beta1"]
    beta2 = kwargs["beta2"]
    momentum = kwargs["momentum"]

    netBreastCancer = Model()

    if adam:
        optimizer = optim.Adam(
            params=netBreastCancer.parameters(), lr=lr, betas=(beta1, beta2)
        )

    elif SGD:
        optimizer = optim.SGD(
            params=netBreastCancer.parameters(), lr=lr, momentum=momentum
        )

    criterion = nn.BCELoss(reduction="mean")

    dataloader = load_dataloader()

    return {
        "model": netBreastCancer,
        "optimizer": optimizer,
        "criterion": criterion,
        "train_dataloader": dataloader["train_dataloader"],
        "test_dataloader": dataloader["test_dataloader"],
    }


if __name__ == "__main__":
    init = helpers(
        lr=0.01,
        adam=True,
        SGD=False,
        beta1=0.9,
        beta2=0.5,
        momentum=0.9,
    )

    print(init["model"])
    print(init["train_dataloader"])
    print(init["test_dataloader"])

    assert init["optimizer"].__class__.__name__ == "Adam"
    assert init["criterion"].__class__.__name__ == "BCELoss"

In [ ]:
import os
import torch
import mlflow
import argparse
import numpy as np
from tqdm import tqdm
from utils import config, dump, load
from helper import helpers
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


class Trainer:
    def __init__(
        self,
        epochs=100,
        lr=0.01,
        beta1=0.5,
        beta2=0.999,
        momentum=0.9,
        step_size=7,
        gamma=0.1,
        adam=True,
        SGD=False,
        device="cpu",
        is_display=True,
        lr_scheduler=False,
    ):
        self.epochs = epochs
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.adam = adam
        self.SGD = SGD
        self.momentum = momentum
        self.step_size = step_size
        self.gamma = gamma
        self.device = device
        self.is_display = is_display
        self.lr_scheduler = lr_scheduler

        self.experiment_name = "Breast Cancer Classification"

        self.init = helpers(
            lr=self.lr,
            beta1=self.beta1,
            beta2=self.beta2,
            momentum=self.momentum,
            adam=self.adam,
            SGD=self.SGD,
        )

        self.train_dataloader = self.init["train_dataloader"]
        self.valid_dataloader = self.init["test_dataloader"]

        self.model = self.init["model"].to(self.device)
        self.criterion = self.init["criterion"]
        self.optimizer = self.init["optimizer"]

        if self.lr_scheduler:
            self.scheduler = StepLR(
                optimizer=self.optimizer, step_size=self.step_size, gamma=self.gamma
            )

        self.history = {"train_loss": [], "test_loss": []}
        self.loss = float("inf")
        self.config = config()

        self.raw_data_path = self.config["path"]["RAW_PATH"]
        self.files_path = self.config["path"]["FILES_PATH"]
        self.train_models_path = self.config["path"]["TRAIN_MODEL_PATH"]
        self.best_model_path = self.config["path"]["BEST_MODEL_PATH"]

    def update_model(self, X, y):
        if isinstance(X, torch.Tensor) and isinstance(y, torch.Tensor):
            X = X.float()
            y = y.float()

            self.optimizer.zero_grad()

            predicted = self.model(X)
            predicted_loss = self.criterion(predicted, y)

            predicted_loss.backward()
            self.optimizer.step()

            return predicted, predicted_loss.item()

        else:
            raise TypeError("X and y must be torch.Tensor")

    def display_progress(self, **kwargs):
        if self.is_display:
            print(
                "Epochs - [{}/{}] - Train Loss: {:.4f} - Test Loss: {:.4f} - train_accuracy: {:.4f} - test_accuracy: {:.4f}".format(
                    kwargs["epoch"] + 1,
                    self.epochs,
                    np.mean(kwargs["train_loss"]),
                    np.mean(kwargs["test_loss"]),
                    accuracy_score(kwargs["train_actual"], kwargs["train_pred"]),
                    accuracy_score(kwargs["valid_actual"], kwargs["valid_pred"]),
                )
            )
        else:
            print(
                "Epochs - [{}/{}] is completed".format(kwargs["epoch"] + 1, self.epochs)
            )

    def saved_models(self, epoch=None, train_loss=None):
        torch.save(
            self.model.state_dict(),
            os.path.join(self.train_models_path, "model{}.pth".format(epoch)),
        )

        if train_loss is not None:
            if self.loss > train_loss:
                self.loss = train_loss
                torch.save(
                    {
                        "model": self.model.state_dict(),
                        "loss": train_loss,
                        "epoch": epoch,
                    },
                    os.path.join(self.best_model_path, "best_model.pth"),
                )
        else:
            raise ValueError("Train loss cannot be None".capitalize())

    def compute_model_performace(self, actual, predicted):
        return {
            "accuracy": accuracy_score(actual, predicted),
            "precision": precision_score(actual, predicted),
            "recall": recall_score(actual, predicted),
            "f1_score": f1_score(actual, predicted),
        }

    def train(self):
        mlflow.set_experiment(self.experiment_name)

        with mlflow.start_run(
            description="This is used for to track the Breast Cancer Classification Model loss"
        ) as run:

            for epoch in tqdm(range(self.epochs)):
                train_predcit = []
                valid_predict = []
                train_actual = []
                valid_actual = []
                train_loss = []
                test_loss = []

                for _, (X, y) in enumerate(self.train_dataloader):
                    X, y = X.to(self.device), y.to(self.device)
                    X = X.float()
                    y = y.float()

                    predicted, loss = self.update_model(X=X, y=y.unsqueeze(1))

                    train_predcit.append(
                        torch.where(predicted > 0.5, 1, 0).detach().cpu().numpy()
                    )
                    train_actual.append(
                        torch.where(y > 0.5, 1, 0).detach().cpu().numpy()
                    )

                    train_loss.append(loss)

                for _, (X, y) in enumerate(self.valid_dataloader):
                    X, y = X.to(self.device), y.to(self.device)
                    X = X.float()
                    y = y.float()

                    predicted = self.model(X)
                    loss = self.criterion(predicted, y.unsqueeze(1))

                    valid_predict.append(
                        torch.where(predicted > 0.5, 1, 0).detach().cpu().numpy()
                    )
                    valid_actual.append(
                        torch.where(y > 0.5, 1, 0).detach().cpu().numpy()
                    )

                    test_loss.append(loss.item())

                if self.lr_scheduler:
                    self.scheduler.step()

                train_actual = np.concatenate(train_actual)
                train_predcit = np.concatenate(train_predcit)
                valid_actual = np.concatenate(valid_actual)
                valid_predict = np.concatenate(valid_predict)

                self.display_progress(
                    epoch=epoch,
                    train_loss=train_loss,
                    test_loss=test_loss,
                    train_pred=train_predcit,
                    train_actual=train_actual,
                    valid_pred=valid_predict,
                    valid_actual=valid_actual,
                )

                self.saved_models(epoch=epoch + 1, train_loss=np.mean(train_loss))

                self.history["train_loss"].append(np.mean(train_loss))
                self.history["test_loss"].append(np.mean(test_loss))

                train_performance = self.compute_model_performace(
                    actual=train_actual, predicted=train_predcit
                )
                valid_performance = self.compute_model_performace(
                    actual=valid_actual, predicted=valid_predict
                )

                for index, performace in enumerate(
                    [train_performance, valid_performance]
                ):
                    for metric, value in performace.items():
                        mlflow.log_metric(
                            key="train_" + metric if index == 0 else "valid_" + metric,
                            value=value,
                            step=epoch + 1,
                        )

                for type, loss in [
                    ("train_loss", np.mean(train_loss)),
                    ("test_loss", np.mean(test_loss)),
                ]:
                    mlflow.log_metric(key=type, value=loss, step=epoch + 1)

            print("Training is completed".title())

            dump(
                value=self.history,
                filename=os.path.join(config()["path"]["FILES_PATH"], "history.pkl"),
            )

            mlflow.log_params(
                {
                    "epochs": self.epochs,
                    "lr": self.lr,
                    "beta1": self.beta1,
                    "beta2": self.beta2,
                    "adam": self.adam,
                    "SGD": self.SGD,
                    "momentun": self.momentum,
                    "device": self.device,
                    "display": self.is_display,
                }
            )

            mlflow.pytorch.log_model(self.model, "Breast_Cancer_Model")

            mlflow.log_artifact(
                os.path.join(self.files_path, "Model.jpeg"),
            )

            mlflow.log_artifacts(
                os.path.join(
                    self.raw_data_path,
                ),
                artifact_path="dataset",
            )

    @staticmethod
    def plot_history():
        plt.figure(figsize=(10, 5))

        history = load(
            filename=os.path.join(config()["path"]["FILES_PATH"], "history.pkl")
        )

        for filename, loss in history.items():
            plt.plot(loss, label=filename)
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.legend()

        plt.tight_layout
        plt.savefig(os.path.join(config()["path"]["FILES_PATH"], "loss.png"))
        plt.show()


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Trainer code for Breast Cancer".title()
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=config()["trainer"]["epochs"],
        help="Number of epochs".capitalize(),
    )
    parser.add_argument(
        "--lr",
        type=float,
        default=config()["trainer"]["lr"],
        help="Learning rate".capitalize(),
    )
    parser.add_argument(
        "--beta1",
        type=float,
        default=config()["trainer"]["beta1"],
        help="Beta1".capitalize(),
    )
    parser.add_argument(
        "--beta2",
        type=float,
        default=config()["trainer"]["beta2"],
        help="Beta2".capitalize(),
    )
    parser.add_argument(
        "--adam",
        type=bool,
        default=config()["trainer"]["adam"],
        help="Adam".capitalize(),
    )
    parser.add_argument(
        "--SGD",
        type=bool,
        default=config()["trainer"]["SGD"],
        help="SGD".capitalize(),
    )
    parser.add_argument(
        "--display",
        type=bool,
        default=config()["trainer"]["display"],
        help="Display".capitalize(),
    )

    args = parser.parse_args()

    trainer = Trainer(
        epochs=args.epochs,
        lr=args.lr,
        beta1=args.beta1,
        beta2=args.beta2,
        adam=args.adam,
        SGD=args.SGD,
        is_display=args.display,
    )

    trainer.train()

    trainer.plot_history()

In [ ]:
import os
import torch
import argparse
from utils import load, config
from model import Model
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score,
)


class TestModel:
    def __init__(self):

        self.model = Model()

    def load_dataloader(self):
        return load(
            filename=os.path.join(
                config()["path"]["PROCESSED_PATH"], "test_dataloader.pkl"
            )
        )

    def select_best_model(self):
        if os.path.exists(config()["path"]["BEST_MODEL_PATH"]):
            best_model_path = config()["path"]["BEST_MODEL_PATH"]

            best_model = torch.load(os.path.join(best_model_path, "best_model.pth"))

            self.model.load_state_dict(best_model["model"])

    def saved_performace(self, **kwargs):
        with open(
            os.path.join(config()["path"]["OUTPUTS_PATH"], "valid_performace.txt"), "w"
        ) as file:
            file.write(
                "Accuracy: {}\nPrecision:{}\nRecall:{}\nF1_Score:{}\n{}".format(
                    str(kwargs["accuracy"]),
                    str(kwargs["precision"]),
                    str(kwargs["recall"]),
                    str(kwargs["f1_score"]),
                    "*" * 100,
                )
            )
            file.write(
                "\nConfusion Metrics\n{}\n{}".format(
                    str(confusion_matrix(kwargs["y_true"], kwargs["y_pred"])), "*" * 100
                )
            )
            file.write(
                "\nClassification report\n\n{}:\n".format(
                    str(classification_report(kwargs["y_true"], kwargs["y_pred"]))
                )
            )

    def test(self):
        dataloader = self.load_dataloader()

        print(dataloader)

        self.select_best_model()

        self.predicted = []
        self.actual = []

        for X, y in dataloader:
            X = X.float()
            y = y.float()

            predicted = self.model(X)
            predicted = predicted.view(-1)
            predicted = torch.where(predicted > 0.5, 1, 0)
            predicted = predicted.detach().flatten()

            self.actual.extend(y.detach().flatten())
            self.predicted.extend(predicted)

        accuracy = accuracy_score(self.predicted, self.actual)
        precision = precision_score(self.predicted, self.actual)
        recall = recall_score(self.predicted, self.actual)
        f1 = f1_score(self.predicted, self.actual)

        confusion = confusion_matrix(self.predicted, self.actual)
        classification = classification_report(self.predicted, self.actual)

        self.saved_performace(
            accuracy=accuracy,
            precision=precision,
            recall=recall,
            f1_score=f1,
            y_true=self.actual,
            y_pred=self.predicted,
        )

        print(f"Test Accuracy: {accuracy}")
        print(f"Test Precision: {precision}")
        print(f"Test Recall: {recall}")
        print(f"Test F1 Score: {f1}")

        print("\n")

        print(f"Confusion Matrix: \n {confusion}")
        print("\n")

        print(f"Classification Report: \n {classification}")

        print(
            "Model performace saved in the folder {}".format(
                config()["path"]["OUTPUTS_PATH"]
            )
        )


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Test Model for Breast Cancer".title())
    parser.add_argument("--test", action="store_true", help="Test Model".capitalize())

    args = parser.parse_args()

    if args.test:
        test = TestModel()

        test.test()

In [ ]:
import os
import argparse
from tester import TestModel
from trainer import Trainer
from dataloader import Loader
from utils import config


def train_test_model(**kwargs):
    if kwargs["train"]:
        loader = Loader(
            dataset=kwargs["dataset"],
            batch_size=kwargs["batch_size"],
            split_size=kwargs["split_size"],
        )
        loader.create_dataloader()

        trainer = Trainer(
            epochs=kwargs["epochs"],
            lr=kwargs["lr"],
            beta1=kwargs["beta1"],
            beta2=kwargs["beta2"],
            adam=kwargs["adam"],
            SGD=kwargs["SGD"],
            is_display=kwargs["display"],
        )

        trainer.train()

        trainer.plot_history()

    elif kwargs["test"]:
        test = TestModel()
        test.test()


def cli():
    parser = argparse.ArgumentParser(description="CLI for the project".capitalize())

    parser = argparse.ArgumentParser(
        description="Data Loader for the Cancer".capitalize()
    )
    parser.add_argument(
        "--dataset",
        default=os.path.join(config()["path"]["RAW_PATH"], "breast-cancer.csv"),
        help="Defin the dataset".capitalize(),
    )
    parser.add_argument(
        "--split_size",
        default=config()["data"]["split_size"],
        type=float,
        help="Defin the split size".capitalize(),
    )
    parser.add_argument(
        "--batch_size",
        default=config()["data"]["batch_size"],
        type=int,
        help="Defin the batch size".capitalize(),
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=config()["trainer"]["epochs"],
        help="Number of epochs".capitalize(),
    )
    parser.add_argument(
        "--lr",
        type=float,
        default=config()["trainer"]["lr"],
        help="Learning rate".capitalize(),
    )
    parser.add_argument(
        "--beta1",
        type=float,
        default=config()["trainer"]["beta1"],
        help="Beta1".capitalize(),
    )
    parser.add_argument(
        "--beta2",
        type=float,
        default=config()["trainer"]["beta2"],
        help="Beta2".capitalize(),
    )
    parser.add_argument(
        "--adam",
        type=bool,
        default=config()["trainer"]["adam"],
        help="Adam".capitalize(),
    )
    parser.add_argument(
        "--SGD",
        type=bool,
        default=config()["trainer"]["SGD"],
        help="SGD".capitalize(),
    )
    parser.add_argument(
        "--display",
        type=bool,
        default=config()["trainer"]["display"],
        help="Display".capitalize(),
    )
    parser.add_argument(
        "--config",
        default=None,
        help="Config file to train and test the model".capitalize(),
    )

    parser.add_argument("--train", action="store_true", help="Train Model".capitalize())
    parser.add_argument("--test", action="store_true", help="Test Model".capitalize())

    args = parser.parse_args()

    if args.config is not None:
        train_test_model(
            train=args.train,
            test=args.test,
            dataset=os.path.join(config()["path"]["RAW_PATH"], "breast-cancer.csv"),
            batch_size=config()["data"]["batch_size"],
            split_size=config()["data"]["split_size"],
            epochs=config()["trainer"]["epochs"],
            lr=config()["trainer"]["lr"],
            beta1=config()["trainer"]["beta1"],
            beta2=config()["trainer"]["beta2"],
            adam=config()["trainer"]["adam"],
            SGD=config()["trainer"]["SGD"],
            display=config()["trainer"]["display"],
        )

    else:
        train_test_model(
            train=args.train,
            test=args.test,
            dataset=args.dataset,
            batch_size=args.batch_size,
            split_size=args.split_size,
            epochs=args.epochs,
            lr=args.lr,
            beta1=args.beta1,
            beta2=args.beta2,
            adam=args.adam,
            SGD=args.SGD,
            display=args.display,
        )


if __name__ == "__main__":
    cli()